# LangChain-based Course Planning System

This notebook implements an intelligent course planning system that:
- Analyzes whether a topic is broad or narrow
- Generates a single course for narrow topics
- Splits broad topics into multiple logically structured courses (max 8)
- Orders courses from beginner to advanced
- Ensures no overlap between courses
- Returns structured output using Pydantic models

## Install Required Packages

In [7]:
!pip install langchain langchain-ollama pydantic python-dotenv -q


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Dependencies

In [8]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import json

## Define Pydantic Models

These models ensure strict validation of the output structure.

In [9]:
class Course(BaseModel):
    """Represents a single course in the learning path."""
    course_name: str = Field(..., description="Name of the course")
    description: str = Field(..., description="Detailed description of the course")
    difficulty: str = Field(..., description="Difficulty level: Beginner, Intermediate, or Advanced")
    prerequisites: List[str] = Field(default_factory=list, description="List of prerequisite courses")
    
    @field_validator('difficulty')
    @classmethod
    def validate_difficulty(cls, v: str) -> str:
        """Ensure difficulty is one of the allowed values."""
        allowed = ['Beginner', 'Intermediate', 'Advanced']
        if v not in allowed:
            raise ValueError(f'Difficulty must be one of {allowed}')
        return v


class CoursePlan(BaseModel):
    """Represents a complete course plan with one or more courses."""
    is_broad: bool = Field(..., description="Whether the topic is broad and requires multiple courses")
    total_courses: int = Field(..., description="Total number of courses in the plan")
    courses: List[Course] = Field(..., description="List of courses in the plan")
    
    @field_validator('total_courses')
    @classmethod
    def validate_total_courses(cls, v: int, info) -> int:
        """Ensure total_courses matches the length of courses list."""
        if 'courses' in info.data:
            actual_count = len(info.data['courses'])
            if v != actual_count:
                raise ValueError(f'total_courses ({v}) must equal the number of courses ({actual_count})')
        return v
    
    @field_validator('courses')
    @classmethod
    def validate_max_courses(cls, v: List[Course]) -> List[Course]:
        """Ensure maximum 8 courses."""
        if len(v) > 8:
            raise ValueError('Maximum 8 courses allowed')
        if len(v) < 1:
            raise ValueError('At least 1 course is required')
        return v

## Create Course Planning System

This class uses LangChain with structured output to generate course plans.

In [10]:
class CoursePlanner:
    """LangChain-based course planning system."""
    
    def __init__(self, model_name: str = "phi3:mini", temperature: float = 0.7):
        """
        Initialize the course planner.
        
        Args:
            model_name: Name of the Ollama model to use
            temperature: Temperature for response generation (0.0-1.0)
        """
        self.llm = ChatOllama(model=model_name, temperature=temperature)
        self.structured_llm = self.llm.with_structured_output(CoursePlan)
        self.prompt = self._create_prompt()
        self.chain = self.prompt | self.structured_llm
    
    def _create_prompt(self) -> ChatPromptTemplate:
        """Create the prompt template for course planning."""
        system_message = """You are an expert curriculum designer and educational consultant.

Your task is to analyze a given topic and create a structured course plan.

ANALYSIS RULES:
1. Determine if the topic is BROAD or NARROW:
   - NARROW: A specific, focused topic that can be covered in a single comprehensive course
     Examples: "Introduction to Python Lists", "CSS Flexbox", "Linear Regression in Machine Learning"
   
   - BROAD: A wide-ranging topic that requires multiple courses to cover comprehensively
     Examples: "Machine Learning", "Web Development", "Data Science", "Artificial Intelligence"

2. For NARROW topics:
   - Create exactly 1 course
   - Set is_broad = false
   - Provide a comprehensive course covering the entire topic
   
3. For BROAD topics:
   - Create 2-8 courses (maximum 8)
   - Set is_broad = true
   - Split the topic into logical, non-overlapping courses
   - Order courses from Beginner → Intermediate → Advanced
   - Ensure each course builds upon previous ones
   - Assign appropriate prerequisites

COURSE STRUCTURE RULES:
- Each course must have: course_name, description, difficulty, prerequisites
- Difficulty MUST be exactly one of: "Beginner", "Intermediate", or "Advanced"
- Prerequisites should reference course names from earlier courses in the plan
- First course should typically be "Beginner" with no prerequisites
- Courses should progress logically in difficulty
- Courses must NOT overlap in content
- Each course should cover a distinct subset of the broader topic

OUTPUT RULES:
- total_courses MUST equal the actual number of courses
- Maximum 8 courses allowed
- Do NOT add any extra fields
- Ensure all required fields are present"""

        user_message = """Create a course plan for the following topic:

Course Title: {course_title}
Course Description: {course_description}

Analyze whether this topic is broad or narrow, then create an appropriate course plan."""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            ("user", user_message)
        ])
    
    def create_course_plan(self, course_title: str, course_description: str) -> CoursePlan:
        """
        Generate a course plan based on the input.
        
        Args:
            course_title: The title of the topic
            course_description: A description of what should be covered
            
        Returns:
            CoursePlan object with structured course information
        """
        result = self.chain.invoke({
            "course_title": course_title,
            "course_description": course_description
        })
        
        return result
    
    def print_course_plan(self, plan: CoursePlan) -> None:
        """
        Pretty print the course plan.
        
        Args:
            plan: CoursePlan object to display
        """
        print("=" * 80)
        print(f"COURSE PLAN ANALYSIS")
        print("=" * 80)
        print(f"Topic is: {'BROAD' if plan.is_broad else 'NARROW'}")
        print(f"Total Courses: {plan.total_courses}")
        print("=" * 80)
        print()
        
        for idx, course in enumerate(plan.courses, 1):
            print(f"📚 COURSE {idx}: {course.course_name}")
            print(f"   Difficulty: {course.difficulty}")
            print(f"   Description: {course.description}")
            if course.prerequisites:
                print(f"   Prerequisites: {', '.join(course.prerequisites)}")
            else:
                print(f"   Prerequisites: None")
            print()
        print("=" * 80)

## Initialize the Course Planner

In [11]:
planner = CoursePlanner(model_name="phi3:mini", temperature=0.7)

## Example 1: Narrow Topic (Single Course)

In [12]:
# Example of a narrow topic that should result in a single course
narrow_plan = planner.create_course_plan(
    course_title="Introduction to Python Decorators",
    course_description="Learn how to use and create decorators in Python, including function decorators, class decorators, and decorator patterns."
)

planner.print_course_plan(narrow_plan)

COURSE PLAN ANALYSIS
Topic is: NARROW
Total Courses: 1

📚 COURSE 1: Python Decorators Fundamentals
   Difficulty: Beginner
   Description: This beginner-level course covers the basics of decorators in Python. Students will learn what decorators are and how to apply them using function and class decorators.
   Prerequisites: None



## Example 2: Broad Topic (Multiple Courses)

In [13]:
# Example of a broad topic that should result in multiple courses
broad_plan = planner.create_course_plan(
    course_title="Machine Learning",
    course_description="Comprehensive coverage of machine learning concepts, algorithms, and applications including supervised learning, unsupervised learning, deep learning, and practical implementations."
)

planner.print_course_plan(broad_plan)

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 4

📚 COURSE 1: Introduction to Machine Learning
   Difficulty: Beginner
   Description: This beginner-level course introduces the fundamental concepts and terminology of machine learning. It covers supervised, unsupervised, and reinforcement learning paradigms.
   Prerequisites: None

📚 COURSE 2: Machine Learning Algorithms
   Difficulty: Intermediate
   Description: Dive into various algorithms that form the backbone of machine learning. This intermediate course focuses on understanding and implementing key techniques such as linear regression, logistic regression, decision trees, k-nearest neighbors (kNN), support vector machines (SVM), Gaussian processes, and neural networks.
   Prerequisites: Introduction to Machine Learning

📚 COURSE 3: Deep Dive into Neural Networks
   Difficulty: Advanced
   Description: An advanced course where students will learn about the architecture of deep neural networks, including convolutional and recu

## Example 3: Another Broad Topic

In [14]:
# Another broad topic example
web_dev_plan = planner.create_course_plan(
    course_title="Full Stack Web Development",
    course_description="Complete guide to becoming a full stack web developer, covering frontend technologies, backend development, databases, deployment, and modern web development practices."
)

planner.print_course_plan(web_dev_plan)

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 5

📚 COURSE 1: Introduction to Web Development
   Difficulty: Beginner
   Description: An overview of web development concepts including HTML, CSS, and the basics of JavaScript. Introduction to responsive design principles.
   Prerequisites: None

📚 COURSE 2: Frontend Frameworks Essentials
   Difficulty: Intermediate
   Description: Learn popular front-end frameworks like React or Angular, understand their architecture and components system. Implement responsive design with CSS preprocessors.
   Prerequisites: None

📚 COURSE 3: Backend Development Basics
   Difficulty: Intermediate
   Description: Introduction to server-side programming languages like Python, PHP or Node.js and setting up a basic web application structure with frameworks such as Django or Express.
   Prerequisites: None

📚 COURSE 4: Database Management for Web Applications
   Difficulty: Intermediate
   Description: An understanding of different databases, SQL queries

## Validate the Output Structure

In [15]:
# Validate the structure of the generated plan
print("Validation Results:")
print("=" * 80)

# Check narrow topic
print(f"\n✓ Narrow topic plan:")
print(f"  - is_broad: {narrow_plan.is_broad} (Expected: False)")
print(f"  - total_courses: {narrow_plan.total_courses} (Expected: 1)")
print(f"  - Actual courses: {len(narrow_plan.courses)}")
print(f"  - Match: {narrow_plan.total_courses == len(narrow_plan.courses)}")

# Check broad topic
print(f"\n✓ Broad topic plan:")
print(f"  - is_broad: {broad_plan.is_broad} (Expected: True)")
print(f"  - total_courses: {broad_plan.total_courses}")
print(f"  - Actual courses: {len(broad_plan.courses)}")
print(f"  - Match: {broad_plan.total_courses == len(broad_plan.courses)}")
print(f"  - Within limit (1-8): {1 <= len(broad_plan.courses) <= 8}")

# Validate difficulties
print(f"\n✓ Difficulty validation:")
valid_difficulties = ['Beginner', 'Intermediate', 'Advanced']
for course in broad_plan.courses:
    is_valid = course.difficulty in valid_difficulties
    print(f"  - {course.course_name}: {course.difficulty} ({'✓' if is_valid else '✗'})")

print("\n" + "=" * 80)

Validation Results:

✓ Narrow topic plan:
  - is_broad: False (Expected: False)
  - total_courses: 1 (Expected: 1)
  - Actual courses: 1
  - Match: True

✓ Broad topic plan:
  - is_broad: True (Expected: True)
  - total_courses: 4
  - Actual courses: 4
  - Match: True
  - Within limit (1-8): True

✓ Difficulty validation:
  - Introduction to Machine Learning: Beginner (✓)
  - Machine Learning Algorithms: Intermediate (✓)
  - Deep Dive into Neural Networks: Advanced (✓)
  - Practical Machine Learning Projects: Advanced (✓)



## Export to JSON

You can export the course plan to JSON format for API integration.

In [16]:
# Convert to JSON
json_output = broad_plan.model_dump_json(indent=2)
print("JSON Output:")
print(json_output)

# You can also convert to dict
dict_output = broad_plan.model_dump()
print("\n\nDictionary Output:")
print(dict_output)

JSON Output:
{
  "is_broad": true,
  "total_courses": 4,
  "courses": [
    {
      "course_name": "Introduction to Machine Learning",
      "description": "This beginner-level course introduces the fundamental concepts and terminology of machine learning. It covers supervised, unsupervised, and reinforcement learning paradigms.",
      "difficulty": "Beginner",
      "prerequisites": []
    },
    {
      "course_name": "Machine Learning Algorithms",
      "description": "Dive into various algorithms that form the backbone of machine learning. This intermediate course focuses on understanding and implementing key techniques such as linear regression, logistic regression, decision trees, k-nearest neighbors (kNN), support vector machines (SVM), Gaussian processes, and neural networks.",
      "difficulty": "Intermediate",
      "prerequisites": [
        "Introduction to Machine Learning"
      ]
    },
    {
      "course_name": "Deep Dive into Neural Networks",
      "description": "

## Interactive Usage

Use this cell to test with your own topics!

In [17]:
# Try your own topic here!
custom_plan = planner.create_course_plan(
    course_title="Your Topic Here",  # Change this
    course_description="Your description here"  # Change this
)

planner.print_course_plan(custom_plan)

COURSE PLAN ANALYSIS
Topic is: NARROW
Total Courses: 1

📚 COURSE 1: Introduction to Python Lists
   Difficulty: Beginner
   Description: This comprehensive beginner course focuses on understanding the concept of lists in Python. Students will learn how to create and manipulate list objects including indexing, slicing, appending elements, extending other sequences into a list, removing items using various methods like pop() or del statement.
   Prerequisites: None

